In [2]:
import os
import gc
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import norm
import json
import enum
import sklearn.preprocessing
import argparse
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *
import pickle
import math
from scipy import interpolate
import random
import time
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor


In [3]:
from matplotlib import rcParams, rc
rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Time New Roman']
fsize = 12
rcParams.update({'font.size': fsize})

In [4]:
def getPositionEncoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P


In [48]:
with open(f'example_MC.p', 'rb') as f:
    df_new = pickle.load(f)

with open(f'example_train.p', 'rb') as f:
    df_train = pickle.load(f)
    df_train = df_train.reset_index()

with open(f'example_valid.p', 'rb') as f:
    df_valid = pickle.load(f)
    df_valid = df_valid.reset_index()


In [49]:
list_output_time = []
list_static = []
L_signal_input = len(df_new['disp_ground'][0])

Ndim_encoding = 2
P_encoding = getPositionEncoding(seq_len=L_signal_input, d=Ndim_encoding, n=10000)
list_input_time = ['disp_ground']+[f'encode_{d}' for d in range(Ndim_encoding)]
# list_input_time = ['disp_ground']

for k in list(df_new):
  if (str(k)[:4] == 'disp') & (str(k)[-1] != 'd'):
    list_output_time.append(k)

  if str(k)[:4] != 'disp':
    list_static.append(k)

print(list_input_time)
print(list_output_time)

['disp_ground', 'encode_0', 'encode_1']
['disp_2', 'disp_3', 'disp_4', 'disp_5', 'disp_6']


In [50]:
max_disp = [np.max(np.abs(df_new[list_output_time[-1]][idx])) for idx in range(len(df_new))]
df_new['max_disp'] = max_disp

for d in range(Ndim_encoding):
  df_new[f'encode_{d}'] = [P_encoding[:,d]]*len(df_new)


In [51]:
L_all, L_out = 500, 100
sliding_step = 600
list_time_series = list_input_time + list_output_time
L_in = L_all-L_out
N_input= len(list_input_time)
N_output= len(list_output_time)
N_static = len(list_static)
N_time_series = len(list_input_time)+len(list_output_time)+1

In [52]:
static_scaler = sklearn.preprocessing.StandardScaler().fit(df_new[list_static].values)
target = np.array([df_new[i][0] for i in list_output_time]).T
target_scaler = sklearn.preprocessing.StandardScaler().fit(target)
real = np.array([df_new[i][0] for i in list_time_series]).T
real_scaler = sklearn.preprocessing.StandardScaler().fit(real)

In [53]:
def prepare_data(x, N1, N2, j = 2):
# N1: L_input, N2: L_output, j: sliding_step between 2 segments
  N0 = len(x)
  xxx = np.stack([x[i:N0 - N1+i+1:j, :] for i in range(N1)], axis=1)
  y = xxx[:,-N2:,-len(list_output_time):]

  return np.array(xxx), np.array(y)

In [54]:
### Retrain model with selected data
def gen_data_AI(df_new, list_idx):
  list_static_data = []
  list_X_data = []
  list_Y_label = []

  for idx in list_idx:
    len_signal = np.min([len(df_new[k][idx]) for k in list_time_series ])
    static = np.array([df_new[k][idx] for k in list_static ]).reshape(1,-1)
    scaled_static = static_scaler.transform(static)
    signal = np.array([df_new[k][idx][:len_signal] for k in list_time_series ]).T
    signal = np.pad(signal,((L_all,0),(0,0)),constant_values=0.0)
    scaled_signal = real_scaler.transform(signal)

    X_data, Y_label = prepare_data(scaled_signal, L_all, L_out, j = sliding_step)
    list_X_data.append(X_data)
    list_Y_label.append(Y_label)
    list_static_data.append(np.repeat(scaled_static, len(Y_label), axis = 0))



  redata = np.concatenate(list_X_data, axis = 0)
  relabel = np.concatenate(list_Y_label, axis = 0)
  restatic = np.concatenate(list_static_data, axis = 0)
  xx = np.zeros((np.shape(redata)[0], np.shape(redata)[1], 1))
  xx[:,:np.shape(restatic)[-1],0]=restatic
  retrain_data = np.concatenate((redata, xx), axis=-1)

  return retrain_data, relabel, restatic

In [55]:
gc.collect(2)
gc.collect(2)

0

In [56]:
def inference_2(model, df_new, idx):
  len_signal = np.min([len(df_new[k][idx]) for k in list_time_series ])
  static = np.array([df_new[k][idx] for k in list_static ]).reshape(1,-1)
  scaled_static = static_scaler.transform(static)
  signal = np.array([df_new[k][idx][:len_signal] for k in list_time_series ]).T
  signal = np.pad(signal,((L_all,0),(0,0)),constant_values=0.0)
  scaled_signal = real_scaler.transform(signal)

  data, label = prepare_data(scaled_signal, L_all, L_out, j = L_out)
  static = np.repeat(scaled_static, len(label), axis = 0)    # static: (N_segment, N_static)

  # combine static to time-varying input
  xx = np.zeros((data.shape[0], L_all, 1))
  xx[:,:N_static,0]=static
  x_input = np.concatenate((data, xx), axis=-1)
  L_signal = (np.shape(data)[0]-1)*L_out

  # initial a long output time-series with zero
  y_predict_all = np.zeros((L_signal+L_all, N_output))

  # Starting prediction loop
  itx = L_all
  for j in range(int(L_signal/L_out)+1):
    x_input[j,:L_all, N_input:N_time_series-1] = y_predict_all[itx-L_all:itx,:]
    y_pred = model.predict(x_input[j:j+1,:,:], batch_size=128, verbose=0)
    y_predict_all[itx-L_out:itx,:] = y_pred[:,:,:]
    itx += L_out


  # Rescale back time-varying output
  y_predict_all = target_scaler.inverse_transform(y_predict_all)

  return y_predict_all

In [57]:
idx_train = np.arange(len(df_train))
train_data, train_label, train_static = gen_data_AI(df_train, idx_train)
idx_valid = np.arange(len(df_valid))
val_data, val_label, val_static = gen_data_AI(df_valid, idx_valid)


In [59]:
all_callbacks = [tf.keras.callbacks.ModelCheckpoint('ex.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)]

In [60]:
def tf_quantile_loss(y, y_pred, tau=0.5):
    error = tf.keras.ops.subtract(y, y_pred)
    loss = tf.keras.ops.mean(tf.keras.ops.maximum(tau * error, (tau - 1) * error))    
    return loss

In [61]:
def train_model(model, train_data, train_label, val_data, val_label, list_lr=[0.001], Nepoch=20):
  t_start = time.time()
  for lr in list_lr:
    adam = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss=tf.keras.losses.mae, optimizer=adam)
    history = model.fit( x=train_data, y=train_label, epochs=Nepoch, batch_size=512,
            validation_data=(val_data, val_label), callbacks=all_callbacks, shuffle=True, verbose = True)

  t_end = time.time()
  t_cal = t_end - t_start
  return model, history, t_cal


In [62]:
def aSSRA():
  output0 = tf.keras.ops.zeros_like(all_inputs[:, L_in:, N_input:N_time_series-1] )
  historical_features = tf.keras.ops.concatenate([all_inputs[:, :L_in, N_input:N_time_series-1], output0], axis=1)
  future_features = all_inputs[:, :, :N_input]
  input_static = tf.keras.ops.expand_dims(all_inputs[:, :N_static, -1] , axis = 1)
  static_features = tf.keras.ops.repeat(input_static, L_all, axis=1)  # add static features to every time steps
  features = tf.keras.ops.concatenate([historical_features, future_features, static_features], axis=2)
  features = tf.keras.layers.Conv1D(filters=32, kernel_size=32, strides=4, padding='same', activation='relu', data_format="channels_last")(features)
  features = tf.keras.layers.GRU(units = 32, return_sequences=True, return_state=False)(features) # inputs = (32, 10, 8) --> outputs: (32, 10, 16)
  features_att = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=32, value_dim=32)(features, features)
  features = features + features_att
  features = tf.keras.ops.transpose(features, axes=[0, 2, 1])
  features = tf.keras.layers.Dense(units=L_out, activation='relu')(features)
  features = tf.keras.ops.transpose(features, axes=[0, 2, 1])
  features = tf.keras.layers.Dense(units=N_output)(features)
  return features


In [63]:
# First training
all_inputs = tf.keras.layers.Input(shape=(L_all, N_time_series,))
outputs = aSSRA()
model = tf.keras.Model(inputs=all_inputs, outputs=outputs)
model, history, t_cal = train_model(model, train_data, train_label, val_data, val_label, Nepoch=100)


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 24s 4s/step - loss: 0.6301 - val_loss: 0.5802
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - loss: 0.5771 - val_loss: 0.5682
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - loss: 0.5670 - val_loss: 0.5532
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - loss: 0.5467 - val_loss: 0.5265
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - loss: 0.5159 - val_loss: 0.4887
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - loss: 0.4740 - val_loss: 0.4487
Epoch 7/100
4/6 ━━━━━━━━━━━━━━━━━━━━ 1s 654ms/step - loss: 0.4395

KeyboardInterrupt: 

In [64]:
%%time
plt.figure(figsize=(10,14))
list_ip = np.random.randint(0,len(df_new), 4)
list_color = ['r', 'orange', 'm', 'g']
for i,ip in enumerate(list_ip):
  plt.subplot(421+i*2)
  y_predict_all = inference_2(model, df_new, ip)
  i_s = 2
  length = len(df_new[list_output_time[i_s]][ip])
  t= np.arange(0,len(df_new[list_output_time[i_s]][ip][200:]))*0.01
  plt.plot(t, y_predict_all[L_all:L_all+length,i_s][200:],list_color[i], linewidth=2)
  plt.plot(t, df_new[list_output_time[i_s]][ip][200:],'--k', alpha = 0.6, linewidth=2)
  plt.legend(['a-SSRA', 'FEM'], loc=0)
  plt.title(f'{ip} sensor {list_output_time[i_s]}', fontsize=12)
  plt.xlabel('Time (s)')
  plt.ylabel('Displacement (m)')
  plt.ylim(-0.01, 0.01)
  plt.grid(True)


  plt.subplot(422+i*2)
  y_predict_all = inference_2(model, df_new, ip)
  length = len(df_new[list_output_time[i_s]][ip])
  t= np.arange(0,len(df_new[list_output_time[i_s]][ip][200:]))*0.01
  t1 = np.random.randint(0,2200)
  t2 = t1+500
  yy = y_predict_all[L_all:L_all+length,i_s][200:]
  plt.plot(t[t1:t2], yy[t1:t2],list_color[i], linewidth=2)
  yy = df_new[list_output_time[i_s]][ip][200:]
  plt.plot(t[t1:t2], yy[t1:t2],'--k', alpha = 0.6, linewidth=2)
  plt.title(f'zoomed-in region {t1/100}-{t2/100}s', fontsize=12)
  plt.xlabel('Time (s)')
  plt.ylabel('Displacement (m)')
  plt.grid(True)
  plt.ylim(-0.01, 0.01)


plt.tight_layout();

CPU times: total: 21.4 s
Wall time: 15.9 s
